### Modules importeren

In [47]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

import os
from loguru import logger # pip install loguru
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

servername = 'LAPTOP-NBUM6TMN' # PAS DIT AAN NAAR JE EIGEN SERVER!!!

# Data inlezen

### Connecties maken

In [78]:
# SQLite Connecties
logger.info('Establishing connection with SQLite databases...')
northwind_sqlite_conn = sqlite3.connect('../data/processed/northwind.sqlite')
adventureworks_sqlite_conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
aenc_sqlite_conn = sqlite3.connect('../data/processed/aenc.sqlite')
logger.success('Connections established!')

# Connect to the SQLite database
conn = sqlite3.connect('../data/processed/adventureworks.sqlite')
cursor = conn.cursor()


2024-05-20 17:14:59.512 | INFO     | __main__:<module>:2 - Establishing connection with SQLite databases...
2024-05-20 17:14:59.514 | SUCCESS  | __main__:<module>:6 - Connections established!


# Dataframes maken

### Person

In [79]:
#Collecting tables
person = pd.read_sql('SELECT * FROM "Person.Person"', adventureworks_sqlite_conn)
businessEntity = pd.read_sql('SELECT * FROM "Person.BusinessEntity"', adventureworks_sqlite_conn)
password = pd.read_sql('SELECT * FROM "Person.Password"', adventureworks_sqlite_conn)
emailAddress = pd.read_sql('SELECT * FROM "Person.EmailAddress"', adventureworks_sqlite_conn)
personPhone = pd.read_sql('SELECT * FROM "Person.PersonPhone"', adventureworks_sqlite_conn)
phoneNumberType = pd.read_sql('SELECT * FROM "Person.PhoneNumberType"', adventureworks_sqlite_conn)
businessEntityContact = pd.read_sql('SELECT * FROM "Person.BusinessEntityContact"', adventureworks_sqlite_conn)
contactType = pd.read_sql('SELECT * FROM "Person.ContactType"', adventureworks_sqlite_conn)

#Merges
person = pd.merge(person, businessEntity, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntity'))
person = pd.merge(person, password, on=['BusinessEntityID'], how='outer', suffixes=('', '_password'))
person = pd.merge(person, emailAddress, on=['BusinessEntityID'], how='outer', suffixes=('', '_emailAddress'))
person = pd.merge(person, personPhone, on=['BusinessEntityID'], how='outer', suffixes=('', '_personPhone'))
person = pd.merge(person, phoneNumberType, on=['PhoneNumberTypeID'], how='outer', suffixes=('', '_phoneNumberType'))
person = pd.merge(person, businessEntityContact, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntityContact'))
person = pd.merge(person, contactType, on=['ContactTypeID'], how='outer', suffixes=('', '_contactType'))

#Get most recent ModifiedDate
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntity'] if pd.to_datetime(row['ModifiedDate_businessEntity']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_password'] if pd.to_datetime(row['ModifiedDate_password']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_emailAddress'] if pd.to_datetime(row['ModifiedDate_emailAddress']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_personPhone'] if pd.to_datetime(row['ModifiedDate_personPhone']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_phoneNumberType'] if pd.to_datetime(row['ModifiedDate_phoneNumberType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntityContact'] if pd.to_datetime(row['ModifiedDate_businessEntityContact']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
person['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_contactType'] if pd.to_datetime(row['ModifiedDate_contactType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
person.drop(columns=['ModifiedDate_businessEntity'], inplace=True)
person.drop(columns=['ModifiedDate_password'], inplace=True)
person.drop(columns=['ModifiedDate_emailAddress'], inplace=True)
person.drop(columns=['ModifiedDate_personPhone'], inplace=True)
person.drop(columns=['ModifiedDate_phoneNumberType'], inplace=True)
person.drop(columns=['ModifiedDate_businessEntityContact'], inplace=True)
person.drop(columns=['ModifiedDate_contactType'], inplace=True)

person

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,...,EmailAddressID,EmailAddress,rowguid_emailAddress,PhoneNumber,PhoneNumberTypeID,Name,PersonID,ContactTypeID,rowguid_businessEntityContact,Name_contactType
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Accounting Manager
1,1510.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1509.0,2.0,AD2071C7-AF42-4015-AEFC-C8A7BBCBC39D,Assistant Sales Agent
2,1518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1517.0,2.0,6CA86E4E-FDAB-4403-96D0-55750F18673B,Assistant Sales Agent
3,1522.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1521.0,2.0,0D85BFF5-7900-4930-A093-253041883930,Assistant Sales Agent
4,1528.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1527.0,2.0,F501A45F-B97F-42DE-BFE0-5A1D6A416239,Assistant Sales Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20889,20773.0,IN,0.0,None,Crystal,None,Guo,None,0.0,None,...,19968.0,crystal18@adventure-works.com,95889DA3-82BC-4ACC-8D0A-23CDBAC2976C,1 (11) 500 555-0171,1.0,Cell,NaN,NaN,NaN,NaN
20890,20774.0,IN,0.0,None,Isabella,F,Richardson,None,2.0,None,...,19969.0,isabella91@adventure-works.com,CE60403C-FB69-4232-811A-7F510A27A391,910-555-0166,1.0,Cell,NaN,NaN,NaN,NaN
20891,20775.0,IN,0.0,None,Crystal,S,He,None,0.0,None,...,19970.0,crystal19@adventure-works.com,F3114A36-B573-4CF4-A212-6AC42261DFCE,813-555-0148,2.0,Home,NaN,NaN,NaN,NaN
20892,20776.0,IN,0.0,None,Crystal,None,Zheng,None,1.0,None,...,19971.0,crystal20@adventure-works.com,287006E5-4CB1-4688-9A2F-479F048F27CE,1 (11) 500 555-0171,2.0,Home,NaN,NaN,NaN,NaN


### Product

In [ ]:
#Collecting tables
product = pd.read_sql('SELECT * FROM "Person.Person"', adventureworks_sqlite_conn)
businessEntity = pd.read_sql('SELECT * FROM "Person.BusinessEntity"', adventureworks_sqlite_conn)
password = pd.read_sql('SELECT * FROM "Person.Password"', adventureworks_sqlite_conn)
emailAddress = pd.read_sql('SELECT * FROM "Person.EmailAddress"', adventureworks_sqlite_conn)
personPhone = pd.read_sql('SELECT * FROM "Person.PersonPhone"', adventureworks_sqlite_conn)
phoneNumberType = pd.read_sql('SELECT * FROM "Person.PhoneNumberType"', adventureworks_sqlite_conn)
businessEntityContact = pd.read_sql('SELECT * FROM "Person.BusinessEntityContact"', adventureworks_sqlite_conn)
contactType = pd.read_sql('SELECT * FROM "Person.ContactType"', adventureworks_sqlite_conn)

#Merges
product = pd.merge(person, businessEntity, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntity'))
product = pd.merge(person, password, on=['BusinessEntityID'], how='outer', suffixes=('', '_password'))
product = pd.merge(person, emailAddress, on=['BusinessEntityID'], how='outer', suffixes=('', '_emailAddress'))
product = pd.merge(person, personPhone, on=['BusinessEntityID'], how='outer', suffixes=('', '_personPhone'))
product = pd.merge(person, phoneNumberType, on=['PhoneNumberTypeID'], how='outer', suffixes=('', '_phoneNumberType'))
product = pd.merge(person, businessEntityContact, on=['BusinessEntityID'], how='outer', suffixes=('', '_businessEntityContact'))
product = pd.merge(person, contactType, on=['ContactTypeID'], how='outer', suffixes=('', '_contactType'))

#Get most recent ModifiedDate
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntity'] if pd.to_datetime(row['ModifiedDate_businessEntity']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_password'] if pd.to_datetime(row['ModifiedDate_password']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_emailAddress'] if pd.to_datetime(row['ModifiedDate_emailAddress']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_personPhone'] if pd.to_datetime(row['ModifiedDate_personPhone']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_phoneNumberType'] if pd.to_datetime(row['ModifiedDate_phoneNumberType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_businessEntityContact'] if pd.to_datetime(row['ModifiedDate_businessEntityContact']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)
product['ModifiedDate'] = person.apply(lambda row: row['ModifiedDate_contactType'] if pd.to_datetime(row['ModifiedDate_contactType']) > pd.to_datetime(row['ModifiedDate']) else row['ModifiedDate'], axis=1)

#Drop redundant ModifiedDates
product.drop(columns=['ModifiedDate_businessEntity'], inplace=True)
product.drop(columns=['ModifiedDate_password'], inplace=True)
product.drop(columns=['ModifiedDate_emailAddress'], inplace=True)
product.drop(columns=['ModifiedDate_personPhone'], inplace=True)
product.drop(columns=['ModifiedDate_phoneNumberType'], inplace=True)
product.drop(columns=['ModifiedDate_businessEntityContact'], inplace=True)
product.drop(columns=['ModifiedDate_contactType'], inplace=True)

product

### Connecties sluiten

In [77]:
northwind_sqlite_conn.close()
adventureworks_sqlite_conn.close()
aenc_sqlite_conn.close()